In [1]:
#STEP 1, OPEN AND READ THE DATASET

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
%matplotlib inline
from matplotlib.pylab import rcParams
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


# modify system:time_start and rename to: DATE FORMAT YYYY-MM--DD
# follow directions from: https://www.youtube.com/watch?v=ngAE0dME9RA

path=r"C:\Users\jesus\Downloads\L8_NDVI.csv"  #to be used with windows and Anaconda
#path="/content/trend_230201_v20 (1).csv"



pd.read_csv(path)
data=pd.read_csv(path)#, parse_dates=['DATE'], index_col=['DATE'])
# apply the dtype attribute
data = pd.read_csv(path, index_col='DATE', parse_dates=True, infer_datetime_format=True).asfreq(freq='16D')

#data['DATE']=pd.to_datetime(data['DATE'], infer_datetime_format=True)
#data=data.set_index(['DATE'])

data = data.fillna(method = 'ffill')

#Determine data for training and RMSE calculation, that will also be used to define parameters.  
#The final test model will be with year 2022, not used here.
data = data[(data.index>'2017-01-01') & (data.index<'2024-12-31')]


print(data.head())
print(data.tail())

result_data_type = data.dtypes

print("Output:")
print(result_data_type)
len(data)


            RADAR
DATE             
2017-01-16  0.818
2017-02-01  0.761
2017-02-17  0.719
2017-03-05  0.712
2017-03-21  0.746
            RADAR
DATE             
2022-10-13  0.844
2022-10-29  0.837
2022-11-14  0.842
2022-11-30  0.849
2022-12-16  0.845
Output:
RADAR    float64
dtype: object


136

In [4]:
# STEP 2 ITERATION AND MODEL REQUIREMENTS AND 

import itertools
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX

#make sure you use a range for iteration, i.e. (60,60) is not a valid parameter
#abc defined by ARIMA 2,0,3

#a=range(1,2)
#b=range(0,1)
#c=range(0,1)
d=range(0,3)
e=range(0,3)
f=range(0,3)
g=range(23,24)  #fixed value of 61, 6-days-observation per year, both
                #fixed value of 30, 12-days-observations per year, ascending or descending

combination=list(itertools.product(d,e,f,g))

rmse=[]
order1=[]

# note below in "pred" variable that "(start=len(train),end=(len(data)-1))" is equal to the range of dates of the observed "test" values
# there might be a bias towards selecting parameters that fit the predict values.  This is not a good idea since we have a bias towards creating a model designed to ONLY to predict the test data. It might not work for other dates setup.
# this might splitting the sample in three: training, test1 and test 2.  A comparison with test2 is a real test, since the model have not seeing these data.

train = data.iloc[:65] # 50%
test = data.iloc[65:] # 50% ( observed values not used by SARIMAX that are used to compare with predicted values, note that are used to calculate RMSE)

for defg in combination:
	try:
		model= SARIMAX(train['RADAR'], order=(5,0,0), seasonal_order= (defg)).fit()
		pred=model.predict(start=len(train),end=(len(data)-1))
		error=np.sqrt(mean_squared_error(test,pred))
		order1.append(defg)
		rmse.append(error)
		
	except:
		continue

results_itertools=pd.DataFrame(index=order1,data=rmse, columns=['RMSE'])



In [3]:
results_itertools.to_csv('RMSE_BEST_MODEL_NDVI1.csv')
#path1="/content/RMSE_BEST_MODEL_TREND.csv"
path1=r"C:\Users\jesus\Downloads\RMSE_BEST_MODEL_NDVI1.csv"  #to be used with windows and Anaconda
results_csv=pd.read_csv(path1)
print(results_csv.head())
print(results_csv.tail())

      Unnamed: 0      RMSE
0  (0, 0, 0, 20)  0.115186
1  (0, 0, 0, 21)  0.115186
2  (0, 0, 0, 22)  0.115186
3  (0, 0, 0, 23)  0.115186
4  (0, 0, 0, 24)  0.115186
        Unnamed: 0      RMSE
130  (2, 2, 2, 20)  0.202565
131  (2, 2, 2, 21)  0.167056
132  (2, 2, 2, 22)  0.044923
133  (2, 2, 2, 23)  0.010724
134  (2, 2, 2, 24)  0.082557
